# Example: Boy's surface

Boy's surface is an immersion of the real projective plane $\mathbb RP^2$ in $\mathbb R^3$. It was found by Werner Boy in 1901. Boy constructed this surface as a counterexample to David Hilbert's assumption that the projective plane cannot be immersed in R^3. It is also famous because the Mathematical Research Institute in Oberwolfach has a large model of Boy's surface. [[1]](https://en.wikipedia.org/wiki/Boy%27s_surface)

<p style="text-align: center;">
    <img title="Boy's surface at MFO" width=400 src="https://upload.wikimedia.org/wikipedia/commons/b/b1/Boyflaeche.JPG" alt="Not Found";">
</p>

## Kusner–Bryant parametrization

Since the model in Oberwolfach displays the so-called Kusner-Bryant parametrization of the Boy's surface, we will use it also in this example. Let $w\in\mathbb C$ with $\vert w\vert\leq 1$ and
\begin{align}
g_1 &= -\frac32\mathrm{Im}\left(\frac{w(1-w^4)}{w^6+\sqrt5w^3-1}\right),\\
g_2 &= -\frac32\mathrm{Re}\left(\frac{w(1+w^4)}{w^6+\sqrt5w^3-1}\right),\\
g_3 &= \mathrm{Im}\left(\frac{1+w^6}{w^6+\sqrt5w^3-1}\right)-\frac12.
\end{align}
Boy's surface is then parametrized by the following cartesian coordiantes
\begin{align}
\begin{pmatrix}x\\y\\z\end{pmatrix} = \frac1{g_1^2+g_2^2+g_3^2}\begin{pmatrix}g_1\\g_2\\g_3\end{pmatrix}.
\end{align}

To model this using ```maniflow``` we first need to model the unit disc $D = \{w\in\mathbb C:\vert w\vert\leq 1\}\subset\mathbb C$.
### The unit disc in $\mathbb C$
To model the unit disc $D$ we can apply a homoeomorphism from $[-1,1]\times[-1,1]$ to $D$. Define $\iota\colon\mathbb R^2\to\mathbb C, (x,y)\mapsto x+iy$. We may apply for example
\begin{align}
[-1,1]\times[-1,1]\ni x\mapsto\begin{cases}0\qquad&\text{if } x=(0,0),\\
\frac{\Vert x\Vert_{\infty}}{\Vert x\Vert_2}\cdot \iota(x)\qquad&\text{otherwise}
\end{cases}
\end{align}
This can be modeled by a ```Grid``` and a ```VertexFunction```:

In [1]:
import numpy as np
from maniflow.mesh.utils import VertexFunction
from maniflow.mesh.parameterized import Grid

@VertexFunction
def unit_disc(vertex):
    if np.all(vertex == 0):
        return vertex

    scal = max([abs(i) for i in vertex]) / np.sqrt(sum([pow(i, 2) for i in vertex]))
    return scal * vertex

square = Grid((-1, 1), (-1, 1), 80, 80)
disc = unit_disc(square)

### The parametrization

Now we can code the parametrization using ```maniflow```:

In [2]:
@VertexFunction
def kusner_bryant(vertex):
    # w = vertex[0] * np.exp(complex(0, vertex[1]))
    w = complex(vertex[0], vertex[1])
    nenner = pow(w, 6) + np.sqrt(5) * pow(w, 3) - 1
    k = 1+pow(w, 6)
    g1 = -(3/2) * ((w-pow(w,5)) / nenner).imag
    g2 = -(3/2) * ((w+pow(w,5)) / nenner).real
    g3 = (k/nenner).imag - 0.5

    scal = 1 / sum([pow(i, 2) for i in [g1, g2, g3]])

    return scal * np.array([g1, g3, g2])

boy = kusner_bryant(disc)

Now that we have ```Mesh``` object that represents Boy's surface we can write it to a file.

In [3]:
from maniflow.mesh.obj import OBJFile

OBJFile.write(boy, "boys_surface.obj")

## Rendering

We can now render Boy's surface and have a look at it. To do this and get a better-looking result we need to invert the orientation of ```boy``` as the normal vectors will point inward at this point and the shader will thus not color the 'correctly'. We can invert the orientation of ```boy``` by just reversing the enumeration of the vertices in each face:

In [4]:
for j in range(len(boy.faces)):
    boy.faces[j].vertices = boy.faces[j].vertices[::-1]

Now we can actually render Boy's surface:

In [5]:
from maniflow.render import SVGPainterRenderer
from maniflow.render.scene import *

camera = Camera(position=(5,5,13),target=(0,-0.7,0))
scene = Scene(camera=camera, width=400, height=400, light=np.array([10, -1.5, 50]))
renderer = SVGPainterRenderer(scene)

boy.setStyle("bw")
image = renderer.render(boy)
image # display the image in the notebook

## Creating an animation

(TODO: add more explanations)

In [6]:
N = 100

scene = Scene(camera=camera, width=720, height=720, light=np.array([10, -1.5, 50]))
renderer = SVGPainterRenderer(scene)

shader = Shader(fill=[247, 9, 89], fill_opacity=250, stroke=[249, 2, 0, 255], stroke_width="0.1",
                back_stroke=[247, 9, 89], back_fill=np.int32([234, 9, 110]), back_opacity=250,
                shininess=300, specular=0.8)

for i in range(1, N + 1):
    square = Grid((-i / N, i / N), (-i / N, i / N), 80, 80)
    for j in range(len(square.faces)):
        square.faces[j].vertices = square.faces[j].vertices[::-1]
    
    disc = unit_disc(square)
    boy = kusner_bryant(disc)
    
    boy.setShader(shader)
    image = renderer.render(boy)
    image.save_svg("render/boy" + str(i) + ".svg")

Using the command
```bash
ffmpeg -i render/boy%1d.svg -vf format=yuv420p boys_surface.gif
```
we obtain the file [boys_surface.gif](boys_surface.gif):
<p style="text-align: center;">
    <img title="Boy's surface" width=500 src="boys_surface.gif" alt="Not Found";">
</p>

Thus we have recreated [this](https://mathcurve.com/surfaces.gb/boy/boyanim4.gif) animation from [this](https://mathcurve.com/surfaces.gb/boy/boy.shtml) website.